# Growth Rule Verifications

In this notebook we provide Python code to verify various claims made in Section 5 of

&nbsp;&nbsp;&nbsp;&nbsp; [GPPSS] *Rotation-invariant web bases from hourglass plabic graphs* ([arXiv: 2306.12501](https://arxiv.org/abs/2306.12501))

by Christian Gaetz, Oliver Pechenik, Stephan Pfannerer, Jessica Striker, and Joshua P. Swanson.

1. Plumbings, appliances, and crystal appliances are implemented
2. The explicit calculations in [GPPSS], Example 5.19 are checked.
3. The 88 short rules and several of the long rules in [GPPSS], Figure 24 are verified to be good degenerations, completing the proof of Theorem 5.25.
4. The additional good degenerations in [GPPSS], Lemma 5.22 are verified, completing its proof.
5. Finitely many of the additional good degenerations in [GPPSS], Lemma 5.24(iv) are checked.
6. The claim in [GPPSS], Lemma 5.26 is verified, completing its proof.
7. The additional crystal appliance constraints for the relevant short rules in [GPPSS], Lemma 5.44 and Lemma 5.48 are verified, completing their proofs.
8. The lex conditions in [GPPSS], Lemma 5.31 are verified, completing its proof, along with the claimed converse in Remark 5.32.

## 1. Crystals, plumbings, and appliances

### Classes and important operations

Implements:

* basic operations on the crystal of words in barred and unbarred letters using the `CrystalWord` class. See [GPPSS], Section 5.3 for details.
* `PlumbingPermutation` class, which in the notation of [GPPSS], Section 5.2 encodes a piece of a plumbing, $\pi_s$.
* `Plumbing` class, which is the whole plumbing $\pi_\bullet$.
* `ApplianceMap` class, which is a piece of an appliance, $\sigma_s$.
* `Appliance` class, which is the whole appliance $\sigma_\bullet$.
* `fromCrystalWord`, which constructs the crystal appliance from [GPPSS], Definition 5.16 associated to an appropriate crystal path.
* `is_good_degeneration`, which implements [GPPSS], Theorem 5.18
* `preserves_monotonicity`, which implements conditions in [GPPSS], Lemma 5.44 and Lemma 5.48(A),(B)

In [1]:
from itertools import zip_longest, product

# global constants

RANK    = 3 # the rank R, this corresponds to the number of rows-1
LETTERS = [l for l in range(1,RANK+2)] + [-l for l in range(1,RANK+2)] # the allowed letters in the crystal words 1, 2, ..., (R+1) and -1, -2, ..., -(R+1)
I       = [i for i in range(1,RANK+1)] # the index set for the crystals 1, 2, ..., R

# crystal raising operator e(i) acting on letter l:
def e(l, i):
    assert i in I, f"Index must be in the indexset I = {I}"
    
    if l == i+1:
        return i
    
    if l == -i:
        return -(i+1)
    
    return None

# crystal raising operator f(i) acting on letter l:
def f(l, i):
    assert i in I, f"Index must be in the indexset I = {I}"
    
    if l == i:
        return i+1
    
    if l == -(i+1):
        return -i
    
    return None

# gives the tilde letter of l
def tilde_letter(l):
    return l if l>0 else RANK+2+l

class CrystalWord:
    def __init__(self, word):
        assert all(l in LETTERS for l in word), f"Only letters in {LETTERS} are allowed"
        self.word = word[:]
        
    def __str__(self):
        return self.word.__str__()
    
    def __repr__(self):
        return self.word.__repr__()
    
    def __len__(self):
        return len(self.word)
    
    def __getitem__(self, key):
        return self.word[key]
    
    # returns the position of the left-most unmatched [ and the position of the right-most unmatched ] in the bracketing rule 
    # See [PromPerms] Section 8.3. 
    def unmatched_bracket_positions(self, i):
        assert i in I, f"Index must be in the indexset I = {I}"
        unmatched_openers = []
        unmatched_closers = []
        for idx, l in enumerate(self.word):
            
            # an opener at position idx
            if l == i or l == -(i+1):
                unmatched_openers.append(idx)
                
            # an closer at position idx
            if l == -i or l == i+1:
                if unmatched_openers:
                    unmatched_openers.pop()
                else:
                    unmatched_closers.append(idx)
        
        leftmost_unmatched_opener  = None if len(unmatched_openers) == 0 else unmatched_openers[0]
        rightmost_unmatched_closer = None if len(unmatched_closers) == 0 else unmatched_closers[-1]
        
        return (leftmost_unmatched_opener, rightmost_unmatched_closer)
        
    # applies the crystal raising operator e(i) to the word and returns the new word and if return_position=True  the position where raising operator was acting
    def e(self, i, return_position = False):
        assert i in I, f"Index must be in the indexset I = {I}"
        _, idx = self.unmatched_bracket_positions(i)
        if idx is not None:
            new_word = self.word.copy()
            new_word[idx] = e(new_word[idx], i)
            if return_position:
                return CrystalWord(new_word), idx
            else:
                return CrystalWord(new_word)
        
        if return_position:
            return None, None
        else:
            return None
    
    # applies the crystal lowering operator f(i) to the word and returns the new word and if return_position=True the position where lowering operator was acting
    def f(self, i, return_position = False):
        assert i in I, f"Index must be in the indexset I = {I}"
        idx, _ = self.unmatched_bracket_positions(i)
        if idx is not None:
            new_word = self.word.copy()
            new_word[idx] = f(new_word[idx], i)
            if return_position:
                return CrystalWord(new_word), idx
            else:
                return CrystalWord(new_word)
        
        if return_position:
            return None, None
        else:
            return None
    
    def es(self, e_path, return_positions = False):
        current_word = self
        positions = []
        for i in e_path:
            current_word, idx = current_word.e(i, return_position = True)
            if current_word is None:
                break
            positions.append(idx)
        if return_positions:
            return current_word, zip(positions, e_path)
        else:
            return current_word
        
    def fs(self, f_path, return_positions = False):
        current_word = self
        positions = []
        for i in f_path:
            current_word, idx = current_word.f(i, return_position = True)
            if current_word is None:
                break
            positions.append(idx)
        if return_positions:
            return current_word, zip(positions, f_path)
        else:
            return current_word
    
    # returns true if the word is a highest weight word, else false
    def is_heighest_weight(self):
        return all(self.e(i) is None for i in I)
        
    # returns true if the word is a lowest weight word, else false
    def is_lowest_weight(self):
        return all(self.f(i) is None for i in I)
    
    # cut away the first letter and apply raising operator to get to hwe
    def cut_left(self, return_positions = False):
        assert self.is_heighest_weight(), "word must be a highest weight element"
        new_word = CrystalWord(self.word[1:])
        e_path = next(new_word.all_paths_to_top())
        return new_word.es(e_path, return_positions)
        
    # cut away the last letter and apply lowering operators to get to hwe
    def cut_right(self, return_positions = False):
        assert self.is_lowest_weight(), "word must be a lowest weight element"
        new_word = CrystalWord(self.word[:-1])
        f_path = next(new_word.all_paths_to_bottom())
        return new_word.fs(f_path, return_positions)
    
    # returns all paths to the heighest weight element in lexicographic order
    def all_paths_to_top(self):
        if(self.is_heighest_weight()):
            yield []
        else:
            for i in I:
                new_word = self.e(i)
                if new_word is not None:
                    for p in new_word.all_paths_to_top():
                        yield [i]+p
    
    # returns all paths to the lowest weight element in lexicographic order
    def all_paths_to_bottom(self):
        if(self.is_lowest_weight()):
            yield []
        else:
            for i in I:
                new_word = self.f(i)
                if new_word is not None:
                    for p in new_word.all_paths_to_bottom():
                        yield [i]+p
                        
    def tilde(self):
        return tuple(tilde_letter(l) for l in self)

    def is_tilde_lex_smaller(self, other):
        other = CrystalWord(other[:])
        return self.tilde() < other.tilde()
    
# A plumbing permutation of type (m,n) is a Permutation of {("a",0), ("a",1), ..., ("a", m-1), ("b",0), ("b",1), ..., ("b", n-1)} and is represented as two lists
# of lengths m and n respectively. The first list contains the images ("a",0), ("a",1), ..., ("a", m-1) and the second list the images of ("b",0), ..., ("b", n-1)

class PlumbingPermutation():
    def __init__(self, permutation):
        self.type = len(permutation[0]), len(permutation[1])
        self.a, self.b = permutation[0], permutation[1]
    
    def __repr__(self):
        return [self.a, self.b].__repr__()
    
    def __call__(self, e):
        s, idx = e
        if s == "a":
            return self.a[idx]
        else:
            return self.b[idx]
    def __eq__(self, other):
        return self.a == other.a and self.b == other.b
    
    # Defines horizontal gluing of Plumbing permutations
    def __add__(self, other):
        def offset(e, da, db):
            s, idx = e
            if s == "a":
                return ("a", idx+da)
            else:
                return ("b", idx+db)
            
        a_offset, b_offset = self.type
        a = self.a + [offset(e, a_offset, b_offset) for e in other.a]
        b = self.b + [offset(e, a_offset, b_offset) for e in other.b]
        
        return PlumbingPermutation([a,b])
    
    # Defines the composition of two plumbings self*other
    def __mul__(self, other):
        def swap_ab(e):
            s, idx = e
            if s == "a":
                return ("b", idx)
            else:
                return ("a", idx)
        
        # returning NotImplemented is important so that __rmul__ is triggered in the case (int)*(PlumbingPermutation)
        if not isinstance(other, PlumbingPermutation):
            return NotImplemented
        assert other.type[1] == self.type[0], f"Cannot compose PlumbingPermutations of types {self.type} and {other.type}"
        a = [None for _ in other.a]
        b = [None for _ in self.b]
        for idx in range(len(a)):
            e = ('a', idx)
            while True:
                e = other(e)
                if e[0] == "a":
                    break
                e = swap_ab(e)
                e = self(e)
                if e[0] == "b":
                    break
                e = swap_ab(e)
            a[idx] = e
            
        for idx in range(len(b)):
            e = ('b', idx)
            while True:
                e = self(e)
                if e[0] == "b":
                    break
                e = swap_ab(e)
                e = other(e)
                if e[0] == "a":
                    break
                e = swap_ab(e)
            b[idx] = e
            
        return PlumbingPermutation([a,b])
    
    def __rmul__(self, other):
        assert isinstance(other, int), "Cannot multiply those types"
        return sum((self for _ in range(other-1)), self)
        

class Plumbing():
    def __init__(self, permutations):
        assert len(permutations) == RANK, f"A plumbing consists of {RANK} permutations"
        self.permutations = permutations
        
    def __repr__(self):
        return self.permutations.__repr__()
    
    def __eq__(self, other):
        return all([p==s for p,s in zip(self, other)])
    
    def __getitem__(self, key):
        return self.permutations[key]
    
    def __call__(self, s):
        return self[s-1]
    
    def __mul__(self, other):
        if not isinstance(other, Plumbing):
            return NotImplemented
        return Plumbing([p*s for p,s in zip(self, other)])
    
    def __rmul__(self, other):
        return Plumbing([p*other for p in self])
    
    def __add__(self, other):
        return Plumbing([p+s for p,s in zip(self, other)])
    
    def __rmul__(self, other):
        return Plumbing([other*p for p in self])

# An appliance map is a function from {('b', 0), ('b', 1), ..., ('b', l)} to {('a', 0), ('a', 1), ..., ('a', k), ('b', 0), ('b', 1), ..., ('b', l), ('c', 0), ('c', 1), ..., ('c', m)}
# given as a list containing the images of ('b', 0), ('b', 1), ..., ('b', l)

class ApplianceMap():
    def __init__(self, function):
        self.map = function
        
    def __repr__(self):
        return self.map.__repr__()
        
    def __call__(self, e):
        s, idx = e
        return self.map[idx]
    
    def __eq__(self, other):
        return self.map == other.map
    
    # multiplication of self*other where other is a plumbing permutation
    def __mul__(self, other):
        assert other.type[1] == len(self.map), f"Cannot act with PlumbingPermutations of type {other.type} on ApplianceMap of length {len(self.map)}"
        new_map = [None for _ in other.a]
        for idx in range(len(new_map)):
            e = ('a', idx)
            while True:
                e = other(e)
                if e[0] == "a": # plumbing maps element of A to A.
                    e = ('b', e[1]) # in the appliance set is called B.
                    break
                e = self(e)
                if e[0] != "b": #if appliance maps to A or C
                    break
            new_map[idx] = e
            
        return ApplianceMap(new_map)
    
class Appliance():
    def __init__(self, appliance_maps):
        self.maps = appliance_maps
        
    # creates the crystal appliance as in Definition 5.9
    # word is a Crystal word
    # e_path is a squence of raising operators that bring word to the highest weight element
    # f_path is a squence of lowering operators that bring word to the lowest weight element
    # if transposed is set to true (the default here), the transposed Appliance is returned.
    @classmethod
    def fromCrystalWord(cls, word, e_path, f_path, transposed = True):
        hwe, e_positions = word.es(e_path, return_positions = True)
        lwe, f_positions = word.fs(f_path, return_positions = True)
        assert hwe.is_heighest_weight(), "given e-path doesn't go to highest weight element"
        assert lwe.is_lowest_weight(), "given f-path doesn't go to lowers weight element"
        
        appliance_maps = [[None for _ in range(len(word))] for _ in I]
        
        # upper part (B -> A), phase I:
        for k, pi in enumerate(e_positions):
            p,i = pi
            appliance_maps[i-1][p] = ('a', k)
            
        # lower part (B -> C), phase III:
        for k, pi in enumerate(f_positions):
            p,i = pi
            appliance_maps[i-1][p] = ('c', len(f_path)-k-1)
        
        # middle part (B -> B) upper half, phase II:
        cut = hwe
        for k in range(len(hwe)):
            cut, pos = cut.cut_left(return_positions=True)
            for p, i in pos: # p is the position in the word, i is the index of the raising operator
                appliance_maps[i-1][p+k+1] = ('b', k)
                
        # middle part (B -> B) lower half, phase IV:
        cut = lwe
        for k in range(len(hwe),0,-1):
            cut, pos = cut.cut_right(return_positions=True)
            for p, i in pos: # p is the position in the word, i is the index of the raising operator
                appliance_maps[i-1][p] = ('b', k-1)        
        if transposed:
            return Appliance([ApplianceMap(m) for m in appliance_maps[::-1]])
        else:
            return Appliance([ApplianceMap(m) for m in appliance_maps])
    
    @classmethod
    def crystal_appliances(cls, v, transposed = True):
        v = CrystalWord(v)
        for up,down in product(v.all_paths_to_top(), v.all_paths_to_bottom()):
            yield cls.fromCrystalWord(v, up, down, transposed=transposed)

    def __repr__(self):
        return self.maps.__repr__()
        
    def __eq__(self, other):
        return all(f==g for f,g in zip(self, other))
    
    def __getitem__(self, key):
        return self.maps[key]
    
    def __call__(self, s):
        return self[s-1]
    
    def __mul__(self, other):
        return Appliance([g*p for g,p in zip(self, other)])

def is_good_degeneration(v, w, plumbing):
    v = CrystalWord(v)
    w = CrystalWord(w)
    if any(p!=q for p,q in zip_longest(v.all_paths_to_top(), w.all_paths_to_top())) or any(p!=q for p,q in zip_longest(v.all_paths_to_bottom(), w.all_paths_to_bottom())):
        print("Paths not pairwise equal")
        return False
    
    return all(Appliance.fromCrystalWord(v, up, down) == Appliance.fromCrystalWord(w, up, down)*plumbing for up, down in product(v.all_paths_to_top(), v.all_paths_to_bottom()))

def is_crossing(ac, bd):
    '''Determines if pairs {a,c} and {b,d} from a totally ordered alphabet form a crossing.
       Throws an error if a,b,c,d are not all distinct.'''
    if min(ac) > min(bd):
        ac,bd = bd,ac
    a,c = ac
    b,d = bd
    assert len({a,b,c,d})==4, "Input not all distinct, crossing undefined"
    if a > c:
        a,c = c,a
    if b > d:
        b,d = d,b
    return a<b<c<d

def is_intersecting(ax,by):
    '''Determines if pairs (a,x) and (b,y) from a totally ordered alphabet form an intersection.
       Throws an error if a,b not distinct.'''
    a,x=ax
    b,y=by
    assert a!=b, "Intersection undefined, a=b"
    if x==b or y==a or x==y:
        return True
    return is_crossing(ax,by)

def preserves_monotonicity(v, condition_type):
    '''Assumes the X is at the first two indexes, which are either both positive or the first
       is negative and the second is positive, with witnesses on the right. condition_type is either
       'A' in the first case or 'B' in the second case.'''
    v = CrystalWord(v)
    a = ('b',0)
    b = ('b',1)

    for up, down in product(v.all_paths_to_top(), v.all_paths_to_bottom()):
        rho = Appliance.fromCrystalWord(v, up, down, transposed=False)

        # Check trip2, trip2 conditions
        if rho(2)(a) == b:
            return False
        if is_crossing( (a, rho(2)(a)), (b, rho(2)(b)) ):
            return False
        
        # Check trip1, trip2 conditions
        if condition_type=='A':
            if (is_intersecting( (b, rho(1)(b)), (a, rho(2)(a)) ) or
                is_intersecting( (a, rho(3)(a)), (b, rho(2)(b)) ) or
                is_intersecting( (b, rho(3)(b)), (a, rho(2)(a)) ) or
                is_intersecting( (a, rho(1)(a)), (b, rho(2)(b)) )):
                return False
        
        if condition_type=='B':
            if (is_intersecting( (b, rho(1)(b)), (a, rho(2)(a)) ) or
                is_intersecting( (a, rho(3)(a)), (b, rho(2)(b)) ) or
                is_intersecting( (b, rho(3)(b)), (a, rho(2)(a)) )):
                return False
            
            c = rho(3)(b)
            if c <= b or b[0] != c[0]:
                return False

            for i in range(b[1],c[1]+1):
                t=(b[0],i)
                if is_intersecting( (a, rho(1)(a)), (t, rho(2)(t)) ):
                    return False
    
    return True

# Building blocks for plumbings
bar     = Plumbing([PlumbingPermutation([[('b',0)],[('a',0)]]) for _ in I])

cup     = Plumbing([PlumbingPermutation([[('a',1), ('a',0)],[]]) for _ in I])

X_pp_pp = Plumbing([PlumbingPermutation([[('a',1), ('b',0)], [('b',1), ('a',0)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('b',1), ('a',0)], [('a',1), ('b',0)]])])

X_mm_mm = Plumbing([PlumbingPermutation([[('b',1), ('a',0)], [('a',1), ('b',0)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('a',1), ('b',0)], [('b',1), ('a',0)]])])

X_pp_mm = Plumbing([PlumbingPermutation([[('a',1), ('b',1)], [('a',0), ('b',0)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('b',0), ('a',0)], [('b',1), ('a',1)]])])

X_mm_pp = Plumbing([PlumbingPermutation([[('b',0), ('a',0)], [('b',1), ('a',1)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('a',1), ('b',1)], [('a',0), ('b',0)]])])


X_pm_mp = Plumbing([PlumbingPermutation([[('b',1), ('b',0)], [('a',0), ('a',1)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('b',0), ('b',1)], [('a',1), ('a',0)]])])

X_mp_pm = Plumbing([PlumbingPermutation([[('b',0), ('b',1)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',1), ('a',0)]]),
                    PlumbingPermutation([[('b',1), ('b',0)], [('a',0), ('a',1)]])])

### Example: check [GPPSS], Figure 29

In [2]:
# Create plumbings and appliances
pi         = PlumbingPermutation([[('b',2), ('b',0)], [('a',1),('a',0),('b',1)]])
sigma      = PlumbingPermutation([[('a',3), ('b',0), ('a',1), ('a',0)], [('b',1), ('a',2)]])
pi_sigma   = PlumbingPermutation([[('a',3), ('b',2), ('a',1), ('a',0)], [('a',2), ('b',0), ('b',1)]])
g          = ApplianceMap([('a',0), ('b',2), ('c',1)])
g_pi_sigma = ApplianceMap([('b',3), ('c',1), ('b',1), ('b',0)])

# Check compositions are as claimed
pi_sigma == pi*sigma and g_pi_sigma == g*(pi*sigma) == (g*pi)*sigma

True

## 2. Check {GPPSS], Example 5.19

In [3]:
v  = CrystalWord([-3,2,4])
vp = CrystalWord([-3,-1,-3])

def show_apps(apps,ef='e'):
    print("  Applied:", ", then ".join([f"{ef}_{s} at {p+1}" for p,s in apps]))


print("v =", v)
print()

es = next(v.all_paths_to_top())
assert es == next(vp.all_paths_to_top())

print("Phase I. Raising path: \\vec{e} =", es)

vhw,apps = v.es(es, return_positions=True)
print("  Highest weight:", vhw)
show_apps(apps)
print()

print("Phase II. Upper triangle")
while len(vhw)>0:
    vhw,apps = vhw.cut_left(return_positions=True)
    print("  Highest weight:", vhw)
    show_apps(apps)

fs = next(v.all_paths_to_bottom())
assert fs == next(vp.all_paths_to_bottom())
print()

print("Phase III. Lowering path: \\vec{f} =", fs)

vlw,apps = v.fs(fs, return_positions=True)
print("  Lowest weight:", vlw)
show_apps(apps,'f')
print()

print("Phase IV. Lower triangle")
while len(vlw)>0:
    vlw,apps = vlw.cut_right(return_positions=True)
    print("  Highest weight:", vlw)
    show_apps(apps,'f')
print()

va = Appliance.fromCrystalWord(v, es, fs) # Note: appliances are by default constructed transposed
print(f"Crystal appliance of {v} with {es}, {fs}:")
print("  \\rho_1^\\top =",va(1))
print("  \\rho_2^\\top =",va(2))
print("  \\rho_3^\\top =",va(3))
print("  Note: for Example 5.19, use ('a',0)=1, ('a',1)=2, ('a',2)=3, ('a',3)=4, ('b',0)=5, ('b',1)=6, ('b',2)=7, ('c',0)=8, ('c',1)=9, ('c',2)=10")
print()

vpa = Appliance.fromCrystalWord(vp, es, fs)
pi = bar+X_pp_mm
print(f"Is (\\rho_\\bullet^\\top for {v}) = (\\rho_\\bullet^\\top for {vp}) . \\pi?", va == vpa*pi)

v = [-3, 2, 4]

Phase I. Raising path: \vec{e} = [1, 3, 3, 2]
  Highest weight: [-4, 1, 2]
  Applied: e_1 at 2, then e_3 at 3, then e_3 at 1, then e_2 at 3

Phase II. Upper triangle
  Highest weight: [1, 2]
  Applied: 
  Highest weight: [1]
  Applied: e_1 at 1
  Highest weight: []
  Applied: 

Phase III. Lowering path: \vec{f} = [2, 2, 1]
  Lowest weight: [-1, 3, 4]
  Applied: f_2 at 1, then f_2 at 2, then f_1 at 1

Phase IV. Lower triangle
  Highest weight: [-1, 4]
  Applied: f_3 at 2
  Highest weight: [-1]
  Applied: 
  Highest weight: []
  Applied: 

Crystal appliance of [-3, 2, 4] with [1, 3, 3, 2], [2, 2, 1]:
  \rho_1^\top = [('a', 2), ('b', 2), ('a', 1)]
  \rho_2^\top = [('c', 2), ('c', 1), ('a', 3)]
  \rho_3^\top = [('c', 0), ('a', 0), ('b', 1)]
  Note: for Example 5.19, use ('a',0)=1, ('a',1)=2, ('a',2)=3, ('a',3)=4, ('b',0)=5, ('b',1)=6, ('b',2)=7, ('c',0)=8, ('c',1)=9, ('c',2)=10

Is (\rho_\bullet^\top for [-3, 2, 4]) = (\rho_\bullet^\top for [-3, -1, -3]) . \pi? True


## 3. Verify short and long rules in Figure 24 are good degenerations


In [4]:
# Non example
is_good_degeneration([-4,2],[2,-4], X_mp_pm)

False

In [5]:
# Example
is_good_degeneration([-4,2,4],[2,-4,4], X_mp_pm+bar)

True

### Define the 88 short rules

In [6]:
growth_rules = [
    ([1,-1],[],cup,None),
    ([-4,4],[],cup,None),
    
    ([1,-2],[-2,1],X_pm_mp,'A'),
    ([2,-1],[-1,2],X_pm_mp,'A'),
    ([-3,4],[4,-3],X_mp_pm,None),
    ([-4,3],[3,-4],X_mp_pm,None),
    
    ([2,-2],[-1,1],X_pm_mp,'A'),
    ([-3,3],[4,-4],X_mp_pm,None),
    
    ([1,-3,-1],[-3,1,-1],X_pm_mp+bar,'A'),#
    ([1,-3,2],[-3,1,2],X_pm_mp+bar,'A'),
    ([1,3,-1],[1,-1,3],bar+X_pm_mp,None),#
    ([-2,3,-1],[-2,-1,3],bar+X_pm_mp,None),
    ([-4,-2,4],[-4,4,-2],bar+X_mp_pm,None),#
    ([3,-2,4],[3,4,-2],bar+X_mp_pm,None),
    ([-4,2,4],[2,-4,4],X_mp_pm+bar,None),#
    ([-4,2,-3],[2,-4,-3],X_mp_pm+bar,None),
    
    ([1,2,-1],[2,1,-1],X_pp_pp+bar,'B'),#
    ([1,2,2],[2,1,2],X_pp_pp+bar,'B'),
    ([1,-2,-1],[1,-1,-2],bar+X_mm_mm,None),#
    ([-2,-2,-1],[-2,-1,-2],bar+X_mm_mm,None),
    ([-4,3,4],[-4,4,3],bar+X_pp_pp,None),#
    ([3,3,4],[3,4,3],bar+X_pp_pp,None),
    ([-4,-3,4],[-3,-4,4],X_mm_mm+bar,None),#
    ([-4,-3,-3],[-3,-4,-3],X_mm_mm+bar,None),
    
    ([1,3,-1],[3,1,-1],X_pp_pp+bar,'B'),#
    ([1,3,2],[3,1,2],X_pp_pp+bar,'B'),
    ([1,3,3],[3,1,3],X_pp_pp+bar,'B'),
    ([1,-3,-1],[1,-1,-3],bar+X_mm_mm,None),#
    ([-2,-3,-1],[-2,-1,-3],bar+X_mm_mm,None),
    ([-3,-3,-1],[-3,-1,-3],bar+X_mm_mm,None),
    ([-4,2,4],[-4,4,2],bar+X_pp_pp,None),#
    ([3,2,4],[3,4,2],bar+X_pp_pp,None),
    ([2,2,4],[2,4,2],bar+X_pp_pp,None),
    ([-4,-2,4],[-2,-4,4],X_mm_mm+bar,None),#
    ([-4,-2,-3],[-2,-4,-3],X_mm_mm+bar,None),
    ([-4,-2,-2],[-2,-4,-2],X_mm_mm+bar,None),
    
    ([1,4,-1],[4,1,-1],X_pp_pp+bar,'B'),#
    ([1,4,2],[4,1,2],X_pp_pp+bar,'B'),
    ([1,4,3],[4,1,3],X_pp_pp+bar,'B'),
    ([1,4,4],[4,1,4],X_pp_pp+bar,'B'),
    ([1,-4,-1],[1,-1,-4],bar+X_mm_mm,None),#
    ([-2,-4,-1],[-2,-1,-4],bar+X_mm_mm,None),
    ([-3,-4,-1],[-3,-1,-4],bar+X_mm_mm,None),
    ([-4,-4,-1],[-4,-1,-4],bar+X_mm_mm,None),
    ([-4,1,4],[-4,4,1],bar+X_pp_pp,None),#
    ([3,1,4],[3,4,1],bar+X_pp_pp,None),
    ([2,1,4],[2,4,1],bar+X_pp_pp,None),
    ([1,1,4],[1,4,1],bar+X_pp_pp,None),
    ([-4,-1,4],[-1,-4,4],X_mm_mm+bar,None),#
    ([-4,-1,-3],[-1,-4,-3],X_mm_mm+bar,None),
    ([-4,-1,-2],[-1,-4,-2],X_mm_mm+bar,None),
    ([-4,-1,-1],[-1,-4,-1],X_mm_mm+bar,None),
    
    ([1,2,-3],[-3,-4,-3],X_pp_mm+bar,None),#
    ([1,2,4],[-3,-4,4],X_pp_mm+bar,None),
    ([3,-2,-1],[3,4,3],bar+X_mm_pp,None),#
    ([-4,-2,-1],[-4,4,3],bar+X_mm_pp,None),
    ([-2,3,4],[-2,-1,-2],bar+X_pp_mm,None),#
    ([1,3,4],[1,-1,-2],bar+X_pp_mm,None),
    ([-4,-3,2],[2,1,2],X_mm_pp+bar,'B'),#
    ([-4,-3,-1],[2,1,-1],X_mm_pp+bar,'B'),
    
    ([1,3,-2],[-2,-4,-2],X_pp_mm+bar,None),#
    ([1,3,-3],[-2,-4,-3],X_pp_mm+bar,None),
    ([1,3,4],[-2,-4,4],X_pp_mm+bar,None),
    ([2,-3,-1],[2,4,2],bar+X_mm_pp,None),#
    ([3,-3,-1],[3,4,2],bar+X_mm_pp,None),
    ([-4,-3,-1],[-4,4,2],bar+X_mm_pp,None),
    ([-3,2,4],[-3,-1,-3],bar+X_pp_mm,None),#
    ([-2,2,4],[-2,-1,-3],bar+X_pp_mm,None),
    ([1,2,4],[1,-1,-3],bar+X_pp_mm,None),
    ([-4,-2,3],[3,1,3],X_mm_pp+bar,'B'),#
    ([-4,-2,2],[3,1,2],X_mm_pp+bar,'B'),
    ([-4,-2,-1],[3,1,-1],X_mm_pp+bar,'B'),
    
    ([2,3,-1],[-1,-4,-1],X_pp_mm+bar,None),#
    ([2,3,-2],[-1,-4,-2],X_pp_mm+bar,None),
    ([2,3,-3],[-1,-4,-3],X_pp_mm+bar,None),
    ([2,3,4],[-1,-4,4],X_pp_mm+bar,None),
    ([1,-3,-2],[1,4,1],bar+X_mm_pp,None),#
    ([2,-3,-2],[2,4,1],bar+X_mm_pp,None),
    ([3,-3,-2],[3,4,1],bar+X_mm_pp,None),
    ([-4,-3,-2],[-4,4,1],bar+X_mm_pp,None),
    ([-4,2,3],[-4,-1,-4],bar+X_pp_mm,None),#
    ([-3,2,3],[-3,-1,-4],bar+X_pp_mm,None),
    ([-2,2,3],[-2,-1,-4],bar+X_pp_mm,None),
    ([1,2,3],[1,-1,-4],bar+X_pp_mm,None),
    ([-3,-2,4],[4,1,4],X_mm_pp+bar,'B'),#
    ([-3,-2,3],[4,1,3],X_mm_pp+bar,'B'),
    ([-3,-2,2],[4,1,2],X_mm_pp+bar,'B'),
    ([-3,-2,-1],[4,1,-1],X_mm_pp+bar,'B'),
]

### Verify short rules are good degenerations

In [7]:
all(is_good_degeneration(u,v,p) for u,v,p,_ in growth_rules)

True

### Verify some long rules are good degenerations as well

In [8]:
s = [-3,-2,-3,-3,-2,-2,-2,-3]

long_rules = [
    ([1,4]+s+[-1],[4,1]+s+[-1],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[2],[4,1]+s+[2],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[3],[4,1]+s+[3],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[4],[4,1]+s+[4],X_pp_pp+(len(s)+1)*bar,'B'),
]

all(is_good_degeneration(u,v,p) for u,v,p,_ in long_rules)


True

In [9]:
s = [-3,-2,-2,-3,-3]

long_rules = [
    ([-3,-2]+s+[4],[4,1]+s+[4],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[3],[4,1]+s+[3],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[2],[4,1]+s+[2],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[-1],[4,1]+s+[-1],X_mm_pp+(len(s)+1)*bar,'B'),
]


all(is_good_degeneration(u,v,p) for u,v,p,_ in long_rules)


True

### Verify a a good degeneration which is not part of the growth rules

In [10]:
is_good_degeneration([2,2,3,3],[2,3,2,3],bar+X_pp_pp+bar)

True

## 4. Verify additional good degenerations in [GPPSS], Lemma 5.22

In [11]:
H = X_pm_mp*X_mp_pm

In [12]:
# Sanity check
H*H == 2*bar

True

### Define extra rules in Lemma 5.22

In [13]:
extra_rules = [
    ([-3,2],[2,-3],H),
    ([2,-3],[-3,2],H),
    
    ([-2,2],[3,-3],H),
    ([3,-3],[-2,2],H),
    
    ([-3,-1],[2,4],H),
    ([2,4],[-3,-1],H),
    
    ([-2,-1],[3,4],H),
    ([3,4],[-2,-1],H),
    
    ([-3,-2],[1,4],H),
    ([1,4],[-3,-2],H),
    
    ([-4,-3],[1,2],H),
    ([1,2],[-4,-3],H),
    
    ([1,-2],[-2,1],X_pm_mp),
    ([-2,1],[1,-2],X_pm_mp),
    
    ([2,-1],[-1,2],X_pm_mp),
    ([-1,2],[2,-1],X_pm_mp),
    
    ([2,-2],[-1,1],X_pm_mp),
    ([-1,1],[2,-2],X_pm_mp),
    
    ([-3,4],[4,-3],X_mp_pm),
    ([4,-3],[-3,4],X_mp_pm),
    
    ([-4,3],[3,-4],X_mp_pm),
    ([3,-4],[-4,3],X_mp_pm),
    
    ([-3,3],[4,-4],X_mp_pm),
    ([4,-4],[-3,3],X_mp_pm),
]

### Verify extra rules are good degenerations

In [14]:
all(is_good_degeneration(u,v,p) for u,v,p in extra_rules)

True

## 5. Check some additional good degenerations in Lemma 5.24(iv)

In [15]:
def zeta(k):
    p1 = PlumbingPermutation([[('b',i) for i in range(0,k+3)], [('a',k+2)] + [('a',i) for i in range(1,k+2)] + [('a',0)]])
    p2 = PlumbingPermutation([[('b',1), ('b',0)] + [('b',i) for i in range(2,k+3)], [('a',1), ('a',0)] + [('a',i) for i in range(2,k+3)]])
    p3 = PlumbingPermutation([[('b',k+2)] + [('b',i) for i in range(1,k+2)] + [('b',0)], [('a',i) for i in range(0,k+3)]])
    return Plumbing([p1, p2, p3])

In [16]:
all(is_good_degeneration([4,1]+[-2]*k+[4], [-3,-2]+[-2]*k+[4], zeta(k)) for k in range(7))

True

### Check additional identity action claim

In [17]:
p = lambda k: zeta(k) * (X_mm_pp + (k+1)*bar)
for k in range(7):
    pk = p(k)
    v = [-3, -2] + [-2]*k + [4]
    for g in Appliance.crystal_appliances(v):
        if g*pk != g:
            print('Failure!')
print('Done.')

Done.


## 6. Verify growth rule applicability in Lemma 5.26

In [18]:
ts = {1:[1,-4], 2:[2,-3], 3:[3,-2], 4:[4,-1]}

def is_sublist(S, T):
    '''Determines if the list S is a consecutive sublist of the list T.'''
    for i in range(len(T)-len(S)+1):
        if T[i:i+len(S)] == S:
            return True
    return False

growth_rules_Ls = [g[0] for g in growth_rules]
for w in [[1,1,4], [1,2,3], [1,2,4], [1,3,2], [1,3,3], [1,3,4], [1,4,4], [2,2,4], [2,3,4], [3,2,4]]:
    for v in product(*[ts[i] for i in w]):
        v = list(v)
        if not any(is_sublist(L, v) for L in growth_rules_Ls):
            print('Failure!', v)
print('Done.')

Done.


## 7. Verify monotonicty conditions for short rules in Lemma 5.44 and Lemma 5.48

### Short rules preserve monotonicity

In [19]:
all(preserves_monotonicity(v,condition_type) for _,v,_,condition_type
                                             in growth_rules
                                             if condition_type is not None)

True

### Check some long rules preserve monotonicity

In [20]:
s = [-3,-2,-3,-3,-2,-2,-2,-3]

long_rules = [
    ([1,4]+s+[-1],[4,1]+s+[-1],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[2],[4,1]+s+[2],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[3],[4,1]+s+[3],X_pp_pp+(len(s)+1)*bar,'B'),
    ([1,4]+s+[4],[4,1]+s+[4],X_pp_pp+(len(s)+1)*bar,'B'),
]

all(preserves_monotonicity(v,condition_type) for _,v,_,condition_type in long_rules)


True

In [21]:
s = [-3,-2,-2,-3,-3]

long_rules = [
    ([-3,-2]+s+[4],[4,1]+s+[4],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[3],[4,1]+s+[3],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[2],[4,1]+s+[2],X_mm_pp+(len(s)+1)*bar,'B'),
    ([-3,-2]+s+[-1],[4,1]+s+[-1],X_mm_pp+(len(s)+1)*bar,'B'),
]


all(preserves_monotonicity(v,condition_type) for _,v,_,condition_type in long_rules)


True

## 6. Verify lex conditions in Lemma 5.31 and Remark 5.32

In [29]:
biwords = [CrystalWord([a,b]) for a in LETTERS for b in LETTERS]

def satisfies_niceness(ab, cd, proper_labelings, verbose = False):
    a,b = ab
    c,d = cd
    
    # Condition (i):
    if not (tilde_letter(a)==tilde_letter(b)==tilde_letter(c)==tilde_letter(d) or (tilde_letter(a)<tilde_letter(c) and ab.is_tilde_lex_smaller(cd)) ):
        if verbose:
            print("Condition (i) fails")
        return False
    
    # Condition (ii)
    for labeling in proper_labelings:
        if labeling[0] == ab and cd.is_tilde_lex_smaller(labeling[1]):
            if verbose:
                print("Condition (ii) fails because of {}".format(str(labeling)))
            return False
        
    # Condition (iii)
    for labeling in proper_labelings:
        pq, ts = labeling
        if cd.is_tilde_lex_smaller(ts) or cd.tilde() == ts.tilde():
            if pq.is_tilde_lex_smaller(ab):
                if verbose:
                    print("Condition (iii) fails because of {}".format(str(labeling)))
                return False
    
    # Condition (iv)
    for gamma in range(3):
        for beta in range(gamma, gamma+3):
            for alpha in range(beta, beta+3):
                u = [1]*alpha + [2]*beta + [3]*gamma
                if CrystalWord(u+[c,d]).is_heighest_weight():
                    if not CrystalWord(u+[a,b]).is_heighest_weight():
                        if verbose:
                            print("Condition (iv) fails because of {}".format(u))
                        return False
    
    
    return True

In [30]:
# Compute all possible nice labelings for X_pp_pp
def is_pp_pp_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a!=b and c!=d and a>0 and b>0 and c>0 and d>0 and set(ab) == set(cd)

proper_pp_pp_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_pp_pp_labeling(ab,cd)]

[labeling for labeling in proper_pp_pp_labelings if satisfies_niceness(*labeling, proper_pp_pp_labelings)]



[([1, 2], [2, 1]),
 ([1, 3], [3, 1]),
 ([1, 4], [4, 1]),
 ([2, 4], [4, 2]),
 ([3, 4], [4, 3])]

In [31]:
# Compute all possible nice labelings for X_pp_mm
def is_pp_mm_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a>0 and b>0 and c<0 and d<0 and set([a,b,-c,-d]) == set([1,2,3,4])

proper_pp_mm_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_pp_mm_labeling(ab,cd)]

[labeling for labeling in proper_pp_mm_labelings if satisfies_niceness(*labeling, proper_pp_mm_labelings)]

[([1, 2], [-3, -4]),
 ([1, 3], [-2, -4]),
 ([1, 4], [-2, -3]),
 ([2, 3], [-1, -4]),
 ([2, 4], [-1, -3]),
 ([3, 4], [-1, -2])]

In [32]:
# Compute all possible nice labelings for X_pm_mp
def is_pm_mp_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a!=-c and b!=-d and a>0 and b<0 and c<0 and d>0 and set([a,-c]) == set([-b,d])

proper_pm_mp_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_pm_mp_labeling(ab,cd)]

[labeling for labeling in proper_pm_mp_labelings if satisfies_niceness(*labeling, proper_pm_mp_labelings)]

[([1, -1], [-2, 2]),
 ([1, -2], [-2, 1]),
 ([1, -3], [-3, 1]),
 ([1, -4], [-4, 1]),
 ([2, -1], [-1, 2]),
 ([2, -2], [-1, 1]),
 ([3, -1], [-1, 3]),
 ([4, -1], [-1, 4])]

In [33]:
# Compute all possible nice labelings for X_mp_pm
def is_mp_pm_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a!=-c and b!=-d and a<0 and b>0 and c>0 and d<0 and set([a,-c]) == set([-b,d])

proper_mp_pm_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_mp_pm_labeling(ab,cd)]

[labeling for labeling in proper_mp_pm_labelings if satisfies_niceness(*labeling, proper_mp_pm_labelings)]

[([-1, 4], [4, -1]),
 ([-2, 4], [4, -2]),
 ([-3, 3], [4, -4]),
 ([-3, 4], [4, -3]),
 ([-4, 1], [1, -4]),
 ([-4, 2], [2, -4]),
 ([-4, 3], [3, -4]),
 ([-4, 4], [3, -3])]

In [34]:
# Compute all possible nice labelings for X_mm_pp
def is_mm_pp_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a<0 and b<0 and c>0 and d>0 and set([-a,-b,c,d]) == set([1,2,3,4])

proper_mm_pp_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_mm_pp_labeling(ab,cd)]

[labeling for labeling in proper_mm_pp_labelings if satisfies_niceness(*labeling, proper_mm_pp_labelings)]

[([-2, -1], [4, 3]),
 ([-3, -1], [4, 2]),
 ([-3, -2], [4, 1]),
 ([-4, -1], [3, 2]),
 ([-4, -2], [3, 1]),
 ([-4, -3], [2, 1])]

In [35]:
# Compute all possible nice labelings for X_mm_mm
def is_mm_mm_labeling(ab,cd):
    a, b = ab
    c, d = cd
    return a!=b and c!=d and a<0 and b<0 and c<0 and d<0 and set(ab) == set(cd)

proper_mm_mm_labelings = [(ab,cd) for ab in biwords for cd in biwords if is_mm_mm_labeling(ab,cd)]

[labeling for labeling in proper_mm_mm_labelings if satisfies_niceness(*labeling, proper_mm_mm_labelings)]

[([-2, -1], [-1, -2]),
 ([-3, -1], [-1, -3]),
 ([-4, -1], [-1, -4]),
 ([-4, -2], [-2, -4]),
 ([-4, -3], [-3, -4])]